# Using widgets

In [0]:
dbutils.widgets.text("data_source_param", "")
data_source_val = dbutils.widgets.get("data_source_param")

# Run helpers notebooks

In [0]:
%run "../helpers/configuration"

In [0]:
%run "../helpers/functions"

# Ingest and process raw data

In [0]:
# Define schema with DDL
constructors_schema = "constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"

In [0]:
# Read json file
constructors_df = spark.read.json(f"{raw_folder_path}/constructors.json", schema = constructors_schema)
# display(constructors_df)

In [0]:
# Drop url column
constructors_dropped = constructors_df.drop(constructors_df.url)
# display(constructors_dropped)

In [0]:
from pyspark.sql.functions import lit

# Add new column with ingestion timestamp and rename 2 columns

constructors_renamed = constructors_dropped.withColumnRenamed("constructorId", "constructor_id") \
    .withColumnRenamed("constructorRef", "constructor_ref") \
    .withColumn("data_source", lit(data_source_val))

constructors_final = add_ingestion_date(constructors_renamed)

# display(constructors_final)

In [0]:
# Write df to parquet
constructors_final.write.parquet(f"{processed_folder_path}/constructors", mode = "overwrite")

In [0]:
# Check if data has been properly written
display(spark.read.parquet(f"{processed_folder_path}/constructors"))